In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import binarize
import math
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import datetime
import plotly.express as px

Vector Autoregressive (VAR) is a statistical model used to capture the linear interdependencies among multiple time series variables. It is an extension of the univariate autoregressive (AR) model to multivariate time series data. In a VAR model, each variable is modeled as a linear combination of its own lagged values and the lagged values of other variables in the system.

The main idea behind VAR models is to capture the dynamic relationships among multiple variables over time. They are commonly used in econometrics, finance, and other fields where multiple time series variables influence each other.

In [25]:
df = pd.read_excel("Final_Data.xlsx")
df.head(2)

,Date/Time,Air Quality Index,Carbon Monoxide,Hydrogen Sulphide,Methane,Nitric Oxide,Nitrogen Dioxide,Non-methane Hydrocarbons,Outdoor Air Temperature,Ozone,PM10 Mass,PM2.5 Mass,Relative Humidity,Std. Dev. of Wind Direction,Sulphur Dioxide,Total Hydrocarbons,Total Oxides Of Nitrogen,Wind Direction,Wind Speed
0,2015-01-01,2.405967,0.197200,0.0001,1.9479,0.0022,0.007067,0.06665,2.8167,0.0279,1.0000,2.277767,75.8750,13.79165,0.00120,2.052767,0.009267,220.33335,11.9250
1,2015-01-02,2.308133,0.205567,0.0005,2.0104,0.0114,0.017633,0.09585,-16.1375,0.0186,3.0435,6.213633,80.7083,16.14585,0.00185,2.176400,0.029033,131.31250,10.1146


In [26]:
df.columns

Index(['Date/Time', 'Air Quality Index', 'Carbon Monoxide',
       'Hydrogen Sulphide', 'Methane', 'Nitric Oxide', 'Nitrogen Dioxide',
       'Non-methane Hydrocarbons', 'Outdoor Air Temperature', 'Ozone',
       'PM10 Mass', 'PM2.5 Mass', 'Relative Humidity',
       'Std. Dev. of Wind Direction', 'Sulphur Dioxide', 'Total Hydrocarbons',
       'Total Oxides Of Nitrogen', 'Wind Direction', 'Wind Speed'],
      dtype='object')

In [27]:
# Set the 'Date/Time' column as the DataFrame's index
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df.index = df['Date/Time']
df = df.drop('Date/Time', axis= 1)
df.sample(2)

,Air Quality Index,Carbon Monoxide,Hydrogen Sulphide,Methane,Nitric Oxide,Nitrogen Dioxide,Non-methane Hydrocarbons,Outdoor Air Temperature,Ozone,PM10 Mass,PM2.5 Mass,Relative Humidity,Std. Dev. of Wind Direction,Sulphur Dioxide,Total Hydrocarbons,Total Oxides Of Nitrogen,Wind Direction,Wind Speed
Date/Time,,,,,,,,,,,,,,,,,,
2016-12-02,2.065433,0.315300,0.0009,2.125033,0.009000,0.016733,0.106967,-3.59850,0.0083,NaN,9.638867,86.0000,21.375,0.0001,2.233333,0.025700,161.2292,9.07080
2017-10-13,2.365500,0.330567,0.0007,2.143067,0.028667,0.021500,0.195833,-1.08125,0.0073,NaN,10.944467,74.5417,40.375,0.0003,2.344467,0.050167,164.5625,3.74795


In [28]:
df['Hydrogen Sulphide'] = df['Hydrogen Sulphide'].fillna(value = df['Hydrogen Sulphide'].median())
df['Methane'] = df['Methane'].fillna(value = df['Methane'].median())
df['Relative Humidity'] = df['Relative Humidity'].fillna(value = df['Relative Humidity'].median())
df['Non-methane Hydrocarbons'] = df['Non-methane Hydrocarbons'].fillna(value = df['Non-methane Hydrocarbons'].median())
df = df.drop('PM10 Mass', axis = 1)
df.head(2)

,Air Quality Index,Carbon Monoxide,Hydrogen Sulphide,Methane,Nitric Oxide,Nitrogen Dioxide,Non-methane Hydrocarbons,Outdoor Air Temperature,Ozone,PM2.5 Mass,Relative Humidity,Std. Dev. of Wind Direction,Sulphur Dioxide,Total Hydrocarbons,Total Oxides Of Nitrogen,Wind Direction,Wind Speed
Date/Time,,,,,,,,,,,,,,,,,
2015-01-01,2.405967,0.197200,0.0001,1.9479,0.0022,0.007067,0.06665,2.8167,0.0279,2.277767,75.8750,13.79165,0.00120,2.052767,0.009267,220.33335,11.9250
2015-01-02,2.308133,0.205567,0.0005,2.0104,0.0114,0.017633,0.09585,-16.1375,0.0186,6.213633,80.7083,16.14585,0.00185,2.176400,0.029033,131.31250,10.1146


In [29]:
# Test for stationarity using the Augmented Dickey-Fuller (ADF) test 
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Test for {column}:")
    print(f"ADF Statistic: {result[0]}")
    print(f"P-value: {result[1]}\n")

for column in df.columns:
    adf_test(df[column])


ADF Test for Air Quality Index:
ADF Statistic: -7.016488047253318
P-value: 6.712042163041507e-10

ADF Test for Carbon Monoxide:
ADF Statistic: -6.373988931178002
P-value: 2.307888045463217e-08

ADF Test for Hydrogen Sulphide:
ADF Statistic: -6.322439754231385
P-value: 3.042859865253151e-08

ADF Test for Methane:
ADF Statistic: -3.8002973877169905
P-value: 0.0029042737931499656

ADF Test for Nitric Oxide:
ADF Statistic: -3.6164482486356224
P-value: 0.005454247474442374

ADF Test for Nitrogen Dioxide:
ADF Statistic: -2.855049960904099
P-value: 0.050830331865993644

ADF Test for Non-methane Hydrocarbons:
ADF Statistic: -5.19752335726745
P-value: 8.89502327444382e-06

ADF Test for Outdoor Air Temperature:
ADF Statistic: -2.7428674034336877
P-value: 0.06690951496061469

ADF Test for Ozone:
ADF Statistic: -2.808671871485863
P-value: 0.05704569017896928

ADF Test for PM2.5 Mass:
ADF Statistic: -7.12022166435307
P-value: 3.739248570658771e-10

ADF Test for Relative Humidity:
ADF Statistic: -7.

In [30]:
# # If necessary, difference the time series to make it stationary
# # Replace 'data' with the differenced time series if required
# df_diff = df.diff().dropna()
# df_diff.sample(2)

In [31]:
print(df.dtypes)

Air Quality Index              float64
Carbon Monoxide                float64
Hydrogen Sulphide              float64
Methane                        float64
Nitric Oxide                   float64
Nitrogen Dioxide               float64
Non-methane Hydrocarbons       float64
Outdoor Air Temperature        float64
Ozone                          float64
PM2.5 Mass                     float64
Relative Humidity              float64
Std. Dev. of Wind Direction    float64
Sulphur Dioxide                float64
Total Hydrocarbons             float64
Total Oxides Of Nitrogen       float64
Wind Direction                 float64
Wind Speed                     float64
dtype: object


In [32]:
px.line(df[['Air Quality Index', 'Carbon Monoxide', 'Hydrogen Sulphide', 'Methane', 'Nitric Oxide']])

In [33]:
# Set the proportion of data to use for training (e.g., 0.8 for 80%)
train_proportion = 0.8

n_obs = int(len(df) * train_proportion)
train, test = df[0:n_obs], df[n_obs:]

In [34]:
model = VAR(train)
aic_values = []
for i in range(1, 21):  # Test lag orders from 1 to 20
    result = model.fit(i)
    aic_values.append(result.aic)

# Find the lag order with the lowest AIC value
lag_order = np.argmin(aic_values) + 1
print(f"Optimal lag order: {lag_order}")

c:\Users\Elsa\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



Optimal lag order: 3


In [35]:
model_fitted = model.fit(lag_order)

forecast = model_fitted.forecast(y=train.values[-lag_order:], steps=len(test))
forecast_df = pd.DataFrame(forecast, index=df.index[-len(test):], columns=df.columns)

# Calculate the forecasted AQI
forecasted_aqi = forecast_df['Air Quality Index']

In [36]:
actual_aqi = test['Air Quality Index']
error = rmse(actual_aqi, forecasted_aqi)

aqi_range = df['Air Quality Index'].max() - df['Air Quality Index'].min()
rmse_percentage = (error / aqi_range) * 100


print(f'RMSE: {error}')
print(f'RMSE_%: {rmse_percentage}')

RMSE: 0.8110284164027739
RMSE_%: 13.659042548747106


In [37]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(actual_aqi, forecasted_aqi)
mape = (np.abs((actual_aqi - forecasted_aqi) / actual_aqi).mean()) * 100
print('mae: ', mae)
print('mape: ', mape)

mae:  0.49081489088454167
mape:  16.645887728867567


In [39]:
# Fit the VAR model with the optimal lag order
fitted_model = model.fit(lag_order)
last_observations = df.values[-lag_order:]
forecast = fitted_model.forecast(y=last_observations, steps=7)

# Convert the forecast to a DataFrame with appropriate column names and indices
forecast_df = pd.DataFrame(forecast, columns=df.columns, index=pd.date_range(start='2023-04-01', periods=7, closed='left'))

print(forecast_df)


            Air Quality Index  Carbon Monoxide  Hydrogen Sulphide   Methane  \
2023-04-01           2.242807         0.275007           0.000358  2.190747   
2023-04-02           2.348134         0.273195           0.000483  2.192234   
2023-04-03           2.411713         0.290786           0.000477  2.173980   
2023-04-04           2.430067         0.294416           0.000522  2.181117   
2023-04-05           2.443882         0.287742           0.000519  2.167840   
2023-04-06           2.453776         0.286141           0.000528  2.157156   
2023-04-07           2.467811         0.286606           0.000543  2.151994   

            Nitric Oxide  Nitrogen Dioxide  Non-methane Hydrocarbons  \
2023-04-01      0.012134          0.020923                  0.078410   
2023-04-02      0.012415          0.019786                  0.134414   
2023-04-03      0.013392          0.020572                  0.098294   
2023-04-04      0.013041          0.020393                  0.106696   
2023-04

C:\Users\Elsa\AppData\Local\Temp\ipykernel_29808\845313608.py:7: FutureWarning:

Argument `closed` is deprecated in favor of `inclusive`.



In [22]:
px.line(forecast_df[['Air Quality Index', 'Carbon Monoxide', 'Hydrogen Sulphide', 'Methane', 'Nitric Oxide']])